# Data Validation in Training Pipelines


In this notebook, we will go through the process of validating dataframes in a training pipeline using Pandera.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NatanMish/data_validation/blob/main/notebooks/2_training_pipeline_data_validation.ipynb)


#### Install the required packages and import them to the notebook

In [1]:
!pip install sklearn pandera\[strategies\]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 5.1 MB/s 
     |████████████████████████████████| 389 kB 52.2 MB/s 


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandera as pa
from typing import Optional

#### Load the data

In [4]:
house_data = pd.read_csv('https://github.com/NatanMish/data_validation/blob/a77b247b25c6622ce0c8f8cbc505228161c31a3c/data/train.csv?raw=true')

#### Train basic model
We'll start by setting up a training pipeline using Scikit Learn's native class. We only want to select a few basic features for the purpose of this example, so we'll set up a pipeline step class that will select only those features.

In [5]:
class ChooseFeatures(BaseEstimator):
    def __init__(self, features=None):
        self.features = features
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.features]

In [6]:
feature_names = ['LotArea','YearBuilt','1stFlrSF','2ndFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd', 'LotFrontage']

Now we set up the pipeline and fit it to the data.

In [7]:
pipe = Pipeline([
     ('feature_selection', ChooseFeatures(features=feature_names)),
     ('scaler', StandardScaler()),
     ('rf', RandomForestRegressor())
])

In [8]:
X = house_data
y = house_data.SalePrice
pipe.fit(house_data, y)

ValueError: ignored

Looks like our data has null values and this causes the model to break. Let's take a look at Pandera to see how it can help us with this.

<div>
<img src="https://raw.githubusercontent.com/pandera-dev/pandera/master/docs/source/_static/pandera-banner.png" width="500"/>
</div>

Pandera provides a flexible and expressive API for performing data validation on dataframes to make data processing pipelines more readable and robust. Dataframes contain information that pandera explicitly validates at runtime. This is useful in production-critical data pipelines or reproducible research settings. We'll take a look at these Pandera features:

1. Check the types and properties of columns in a pd.DataFrame or values in a pd.Series.

2. Perform more complex statistical validation like hypothesis testing.

3. Integrate with existing data analysis/processing pipelines via function decorators.

4. Define schema models with the class-based API with pydantic-style syntax and validate dataframes using the typing syntax.

5. Synthesize data from schema objects for property-based testing with pandas data structures.

6. Lazily Validate dataframes so that all validation rules are executed before raising an error.

For more information, see [Pandera's documentation](https://pandera.readthedocs.io/en/latest/).

#### 1. DataFrame Schemas - Type Validation

In [9]:
# We'll add one more feature to make it more interesting
feature_names.append('LotConfig')

In [10]:
# Create a basic schema for the house_data DataFrame to check types for just 2 of the feature
basic_types_schema = pa.DataFrameSchema({
    "LotArea": pa.Column(int),
    "LotConfig": pa.Column(str),
    })

In [11]:
# Validate the house_data DataFrame against the basic_schema
# notice that although we only defined two of the features in the dataframe, and Pandera ignored the rest.
basic_types_schema.validate(house_data[feature_names])

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,LotFrontage,LotConfig
0,8450,2003,856,854,2,3,8,65.0,Inside
1,9600,1976,1262,0,2,3,6,80.0,FR2
2,11250,2001,920,866,2,3,6,68.0,Inside
3,9550,1915,961,756,1,3,7,60.0,Corner
4,14260,2000,1145,1053,2,4,9,84.0,FR2
...,...,...,...,...,...,...,...,...,...
1455,7917,1999,953,694,2,3,7,62.0,Inside
1456,13175,1978,2073,0,2,3,7,85.0,Inside
1457,9042,1941,1188,1152,2,4,9,66.0,Inside
1458,9717,1950,1078,0,1,2,5,68.0,Inside


There is an output from the validation, this means that the data is valid.
There are different ways we can specify the type:
- a string alias, as long as it is recognized by pandas.
- a python type: int, float, double, bool, str
- a numpy data type
- a pandas extension type: it can be an instance (e.g pd.CategoricalDtype([“a”, “b”])) or a class (e.g pandas.CategoricalDtype) if it can be initialized with default values.
- a pandera DataType: it can also be an instance or a class.

In [12]:
# Now let's create a schema that does not fit the data types in house data
bad_types_schema = pa.DataFrameSchema({
    "LotArea": pa.Column(int),
    "LotConfig": pa.Column(float),
})

In [13]:
# The bad schema validation will throw an error
bad_types_schema.validate(house_data[feature_names])

SchemaError: ignored

#### 2. DataFrame Schemas - Value Ranges Validation

In [14]:
# Pandera also allows validating value ranges for numerical columns
value_range_schema = pa.DataFrameSchema({
    "LotArea": pa.Column(int, pa.Check(lambda s: s <= 1000000), nullable=False),
    "YearBuilt": pa.Column(int, pa.Check.in_range(1800, 2022)),
})

In [15]:
# Validate the house_data DataFrame against the value_range_schema
value_range_schema.validate(house_data[feature_names])

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,LotFrontage,LotConfig
0,8450,2003,856,854,2,3,8,65.0,Inside
1,9600,1976,1262,0,2,3,6,80.0,FR2
2,11250,2001,920,866,2,3,6,68.0,Inside
3,9550,1915,961,756,1,3,7,60.0,Corner
4,14260,2000,1145,1053,2,4,9,84.0,FR2
...,...,...,...,...,...,...,...,...,...
1455,7917,1999,953,694,2,3,7,62.0,Inside
1456,13175,1978,2073,0,2,3,7,85.0,Inside
1457,9042,1941,1188,1152,2,4,9,66.0,Inside
1458,9717,1950,1078,0,1,2,5,68.0,Inside


#### 3. DataFrame Schemas - Catch Bad Data

What if instead of breaking on error we want to continue processing the dataframe? or we want to skip the bad data? we can use the `failure_cases` attribute of the error message to capture the bad data indices and the `lazy` argument for going over the entire dataframe instead of failing on the first bad row. We can do that by utilizing a try-except block.

In [32]:
# We'll use a small sample of the data to make the example more clear
sample_data = house_data.sample(n=10)

In [33]:
sample_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
860,861,50,RL,55.0,7642,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,6,2007,WD,Normal,189950
10,11,20,RL,70.0,11200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,129500
275,276,50,RL,55.0,7264,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2009,WD,Normal,205000
1092,1093,50,RL,60.0,8400,Pave,NaN,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,136500
294,295,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2009,WD,Normal,167000
168,169,60,RL,62.0,8244,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,183500
616,617,60,RL,NaN,7861,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,183200
444,445,60,RL,70.0,8750,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,210000
1063,1064,30,RM,50.0,6000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2006,WD,Normal,110500
1300,1301,60,RL,NaN,10762,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,225000


In [34]:
# Create a schema that will fail on the first bad data point
catch_bad_data_schema = pa.DataFrameSchema({
    "LotArea": pa.Column(int, pa.Check(lambda s: s <= 1000000)),
    "YearBuilt": pa.Column(int, pa.Check.in_range(1900,1990)),  # notice that the year built has a restrictive range
})

In [31]:
# Validating the house_data DataFrame against the catch_bad_data_schema will throw an error
catch_bad_data_schema.validate(sample_data[feature_names])

SchemaError: ignored

Now let's use a try except block to catch the bad data indices. This is a common and valid practice in Python called EAFP - "easier to ask for forgiveness than permission" which might not be as well recieved in other languages.

In [35]:
try:
    catch_bad_data_schema.validate(sample_data[feature_names], lazy=True)
except pa.errors.SchemaErrors as e:
    failure_cases = e.failure_cases

# Failure cases is a dataframe of the bad data only
failure_cases.head()

,schema_context,column,check,check_number,failure_case,index
0,Column,YearBuilt,"in_range(1900, 1990)",0,2004,168
1,Column,YearBuilt,"in_range(1900, 1990)",0,2002,616
2,Column,YearBuilt,"in_range(1900, 1990)",0,1994,444
3,Column,YearBuilt,"in_range(1900, 1990)",0,1999,1300


In [36]:
# We can easily filter out the bad data from the original dataframe using the failure_cases dataframe
filtered_df = sample_data[~sample_data.index.isin(failure_cases["index"])]

In [37]:
# Let's see that the filtered data passes the validation test
catch_bad_data_schema.validate(filtered_df[feature_names])

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd,LotFrontage,LotConfig
860,7642,1918,912,514,1,3,7,55.0,Corner
10,11200,1965,1040,0,1,3,5,70.0,Inside
275,7264,1925,952,596,2,3,5,55.0,Inside
1092,8400,1925,1390,304,2,4,7,60.0,Inside
294,9600,1953,1644,0,1,3,7,80.0,Inside
1063,6000,1925,1103,0,1,2,5,50.0,Inside


#### 4. DataFrame Schemas - Validate acceptable categorical values

In [38]:
lot_config_values = ["Inside", "Corner", "CulDSac", "FR3"]

In [39]:
lot_config_values_schema = pa.DataFrameSchema({
    "LotArea": pa.Column(int, pa.Check(lambda s: s <= 1000000)),
    "LotConfig": pa.Column(str, pa.Check.isin(lot_config_values)),
})

In [40]:
# Validating the house_data DataFrame against the lot_config_values_schema will throw an error
lot_config_values_schema.validate(house_data[feature_names])

SchemaError: ignored

Other useful methods for `pa.Check` are:

<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.eq.html">pandera.checks.Check.eq</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.equal_to.html">pandera.checks.Check.equal_to</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.ge.html">pandera.checks.Check.ge</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.greater_than.html">pandera.checks.Check.greater_than</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.greater_than_or_equal_to.html">pandera.checks.Check.greater_than_or_equal_to</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.gt.html">pandera.checks.Check.gt</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.in_range.html">pandera.checks.Check.in_range</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.isin.html">pandera.checks.Check.isin</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.le.html">pandera.checks.Check.le</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.less_than.html">pandera.checks.Check.less_than</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.less_than_or_equal_to.html">pandera.checks.Check.less_than_or_equal_to</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.lt.html">pandera.checks.Check.lt</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.ne.html">pandera.checks.Check.ne</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.not_equal_to.html">pandera.checks.Check.not_equal_to</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.notin.html">pandera.checks.Check.notin</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.str_contains.html">pandera.checks.Check.str_contains</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.str_endswith.html">pandera.checks.Check.str_endswith</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.str_length.html">pandera.checks.Check.str_length</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.str_matches.html">pandera.checks.Check.str_matches</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.str_startswith.html">pandera.checks.Check.str_startswith</a></li>
<li class="toctree-l4"><a class="reference internal" href="methods/pandera.checks.Check.__call__.html">pandera.checks.Check.__call__</a></li>

#### 5. DataFrame Schemas - `Coerce`

`Coerce` allows forcing type onto a specific dataframe column

In [41]:
house_data.LotArea.dtype

dtype('int64')

In [42]:
coerce_schema = pa.DataFrameSchema(
    columns={"LotArea": pa.Column(float)},
    coerce=False,
)

In [43]:
coerce_schema.validate(house_data)

SchemaError: ignored

In [44]:
# and if we set coerce to True, we can coerce the dataframe to the schema
coerce_schema = pa.DataFrameSchema(
    columns={"LotArea": pa.Column(float)},
    coerce=True,
)

In [45]:
coerce_schema.validate(house_data)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717.0,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


#### 6. DataFrame Schemas - `Strict`

In [47]:
# Using `strict` we can specify that the dataframe must have the exact columns specified in the schema
strict_schema = pa.DataFrameSchema(
    columns={"LotArea": pa.Column(int), "YearBuilt": pa.Column(int)},
    strict=True,
)

In [48]:
# Another useful feature is setting `strict` to 'filter' which will filter out any columns that are not in the schema
strict_filter_schema = pa.DataFrameSchema(
    columns={"LotArea": pa.Column(int), "YearBuilt": pa.Column(int)},
    strict="filter",
)
filtered_df = strict_filter_schema.validate(house_data)
filtered_df.head()

,LotArea,YearBuilt
0,8450,2003
1,9600,1976
2,11250,2001
3,9550,1915
4,14260,2000


### Exercise 1 - DataFrame Schemas

Create a pa.DataFrameSchema object for the `house_data` DataFrame. Not all the checks requested were shown above, for some of them you'll need to have a quick search in the Pandera documentation. It should have the following columns and rules:
1. Id is a required and unique column of an integer type and cannot be null.
2. MSZoning is a non-required column of a string type and can be null. If not null it can only accept these values - 'RL', 'RM', 'C (all)', 'RH' and 'FV'.
3. OverallQual is a required column of an integer type, cannot be null and must be in the range 1-10.
4. BsmtCond is a non-required column of a string type and can be null. If not null it can only accept a string of a length of 2.

Bonus:

5. Add the 1stFlrSF and 2ndFlrSF columns to the schema and validate that on average 1stFlrSF>=2ndFlrSF.

Create the schema such that it filters out any other columns that are not in the schema.


In [56]:
exercise_schema = pa.DataFrameSchema(
    columns={
        "Id": pa.Column(int, required=True, nullable=False, unique=True),
        "MSZoning": pa.Column(str, required=False, nullable=True, unique=False, checks=pa.Check.isin(['RL', 'RM', 'C (all)', 'RH', 'FV'])),
        "OverallQual": pa.Column(int, required=True, nullable=False, checks=pa.Check.in_range(1, 10)),
        "BsmtCond": pa.Column(str, required=False, nullable=True, checks=pa.Check.str_length(2)),
        "1stFlrSF": pa.Column(int, nullable=True),
        "2ndFlrSF": pa.Column(int, nullable=True),
    },
    strict="filter",
    checks=pa.Check(lambda x: x["2ndFlrSF"].mean() <= x["1stFlrSF"].mean()),
)

In [57]:
exercise_schema.validate(house_data)

,Id,MSZoning,OverallQual,BsmtCond,1stFlrSF,2ndFlrSF
0,1,RL,7,TA,856,854
1,2,RL,6,TA,1262,0
2,3,RL,7,TA,920,866
3,4,RL,7,Gd,961,756
4,5,RL,8,TA,1145,1053
...,...,...,...,...,...,...
1455,1456,RL,6,TA,953,694
1456,1457,RL,6,TA,2073,0
1457,1458,RL,7,Gd,1188,1152
1458,1459,RL,5,TA,1078,0


*Exercise solutions can be found in the exercise solutions file in the current directory.*

### 7. Pandera Decorators

Pandera offers decorators which allow a seamless integration of Pandera validations with our code. The available decorators are:
- @check_input
- @check_output
- @check_io
- @check_types

We will use a different way for defining the schemas in the next example, but the same principles apply. Here we will construct a class based Pandera model which we can use to validate inputs and outputs to our data in Pydantic style syntax.

In [ ]:
feature_names.remove('LotConfig')

In [ ]:
from pandera.typing import Series

# Define a class based Pandera model for the input data to the feature engineering step
class FeaturesSchemaPreEngineering(pa.SchemaModel):
    LotArea: Series[int] = pa.Field(nullable=False, ge=0)
    YearBuilt: Series[int] = pa.Field(nullable=False, ge=1700)
    FirstFlrSF: Series[int] = pa.Field(nullable=False, ge=0, alias="1stFlrSF") # alias is used to give the column a different name in the schema because the column name starts with a number
    SecondFlrSF: Series[int] = pa.Field(nullable=False, ge=0, alias="2ndFlrSF")
    FullBath: Series[int] = pa.Field(nullable=False, ge=0)
    BedroomAbvGr: Series[int] = pa.Field(nullable=False, ge=0)
    TotRmsAbvGrd: Series[int] = pa.Field(nullable=False, ge=0)
    LotFrontage: Series[float] = pa.Field(nullable=True, ge=0)
    class Config:
        strict=True

# Define a class based Pandera model for the output data to the feature engineering step, notice how we inherit the FeaturesSchemaPreEngineering class and extend it with the output data schema.
class FeaturesSchemaPostEngineering(FeaturesSchemaPreEngineering):
    LotFrontage: Series[float] = pa.Field(nullable=False, ge=0)
    HouseAge: Series[int] = pa.Field(nullable=False, ge=0)
    AllFloorsSF: Series[int] = pa.Field(nullable=False, ge=0)
    NonBedRmAbvGrd: Series[int] = pa.Field(nullable=False, ge=0)

    class Config:
        strict=True

In [ ]:
from pandera import check_types
from pandera.typing import DataFrame as DataFramePa
@check_types
def feature_engineering(df: DataFramePa[FeaturesSchemaPreEngineering]) -> DataFramePa[FeaturesSchemaPostEngineering]:
    df = df.copy()
    df["HouseAge"] = 2022 - df["YearBuilt"]
    df["AllFloorsSF"] = df["1stFlrSF"] + df["2ndFlrSF"]
    df["NonBedRmAbvGrd"] = df["TotRmsAbvGrd"] - df["BedroomAbvGr"]
    return df

In [ ]:
# This run should fail because we haven't filtered out the null values
feature_engineering(house_data.loc[:, feature_names])

In [ ]:
# This run should complete without error
feature_engineering(house_data.loc[~house_data.LotFrontage.isnull(), feature_names])

### 8. Data Synthesis

Pandera offers a simple way to generate synthetic data. We can use the `example` method to generate a DataFrame with a given schema.

In [ ]:
FeaturesSchemaPreEngineering.example(size=5)

Notice how some of the columns have 'crazy' values, this is because the random data generating process is using the checks we have defined in the schema for detecting the acceptable ranges possible.

We can use the hypothesis library to generate data for our schema and then use it in a unit test:

In [ ]:
import hypothesis
@hypothesis.given(FeaturesSchemaPreEngineering.strategy(size=1))
def test_processing_fn(dataframe):
    feature_engineering(dataframe)
    
test_processing_fn()

### 9. Schema Inference

Pandera can infer schemas from data. This is useful when you have a large dataset and you don't want to define a schema manually.

In [ ]:
schema = pa.infer_schema(house_data)
print(schema)

### Exercise 2 - Incorporating validation in a training pipeline

Let's use what we have learned to implement data validation in a training pipeline.
Choose which features to use for training. You can use these, or add your own:

In [ ]:
feature_names = ['LotArea','YearBuilt','1stFlrSF','2ndFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd', 'LotFrontage']

If you chose other features, make sure to include the ones that are required for the schema to be valid.

In [ ]:
# class FeaturesSchemaPreEngineering(pa.SchemaModel):
#     ...

# class FeaturesSchemaPostEngineering(FeaturesSchemaPreEngineering):
#     ...

In [ ]:
@check_types
def feat_eng_step_1(df: DataFramePa[FeaturesSchemaPreEngineering]) -> DataFramePa[FeaturesSchemaPostEngineering]:
    df = df.copy()
    df["HouseAge"] = 2022 - df["YearBuilt"]
    df["AllFloorsSF"] = df["1stFlrSF"] + df["2ndFlrSF"]
    df["NonBedRmAbvGrd"] = df["TotRmsAbvGrd"] - df["BedroomAbvGr"]
    return df

Fill in the missing steps below:
1. First try to run the feat_eng_step_1 function on the X dataframe, and return a tuple of the feature engineered dataframe and the y series.
2. If a pa.errors.SchemaError is raised, capture the exception and extract the `failure_cases` property from the exception. Filter out the invalid data indices from X and y, and then return a tuple comprised of:
    - the filtered X dataframe
    - the filtered y series
    - the failure cases

In [ ]:
def feat_eng_all_steps(X: pd.DataFrame, y: pd.Series) -> (pd.DataFrame, pd.Series, Optional[pd.DataFrame]):
    try:
        <YOUR CODE HERE>
    except <YOUR CODE HERE> as e:
        <YOUR CODE HERE>

Same pipeline as defined previously:

In [ ]:
pipe = Pipeline([
    ('feature_selection', ChooseFeatures(features=feature_names)),
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor())
])

Running the cell below should fit the data to the pipeline, this time with no errors:

In [ ]:
X, y, isolated_invalid_data  = feat_eng_all_steps(house_data[feature_names], house_data["SalePrice"])
pipe.fit(X, y)

In [ ]:
# And we can see the invalid data we were unable to fit:
isolated_invalid_data.head()